In [2]:
!pip install autoawq

  Obtaining dependency information for autoawq from https://files.pythonhosted.org/packages/e0/45/4ec85c794a9436f14b8ed408511300622f6fc4ca8ab58ec7e34ddf5f2496/autoawq-0.1.0-cp310-cp310-manylinux2014_x86_64.whl.metadata
  Using cached autoawq-0.1.0-cp310-cp310-manylinux2014_x86_64.whl.metadata (12 kB)
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/d9/92/2d3aecf9f4a192968035880be3e2fc8b48d541c7128f7c936f430d6f96da/accelerate-0.23.0-py3-none-any.whl.metadata
  Using cached accelerate-0.23.0-py3-none-any.whl.metadata (18 kB)
  Using cached lm_eval-0.3.0-py3-none-any.whl (178 kB)
  Using cached texttable-1.6.7-py2.py3-none-any.whl (10 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached attributedict-0.3.0-py3-none-any.whl (14 kB)
  Obtaining dependency information for protobuf from https://files.pythonhosted.org/packages/bb/c3/6a06208ecf0934ecaf509b51c52a6cf688586f54ae81ac65c56124571494/protobuf-4.24.3-cp37-abi3-manylinu

In [3]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = 'elyza/ELYZA-japanese-Llama-2-7b-instruct'
quant_path = 'ELYZA-japanese-Llama-2-7b-instruct-awq'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4 }

# Load model
model = AutoAWQForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

AWQ: 100% 32/32 [15:13<00:00, 28.54s/it]


('ELYZA-japanese-Llama-2-7b-instruct-awq/tokenizer_config.json',
 'ELYZA-japanese-Llama-2-7b-instruct-awq/special_tokens_map.json',
 'ELYZA-japanese-Llama-2-7b-instruct-awq/tokenizer.model',
 'ELYZA-japanese-Llama-2-7b-instruct-awq/added_tokens.json',
 'ELYZA-japanese-Llama-2-7b-instruct-awq/tokenizer.json')

In [1]:
%%time
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer, TextStreamer
import torch

quant_path = './ELYZA-japanese-Llama-2-7b-instruct-awq'
quant_file = "pytorch_model.bin"

# Load model
model = AutoAWQForCausalLM.from_quantized(quant_path, quant_file, fuse_layers=True)
tokenizer = AutoTokenizer.from_pretrained(quant_path, trust_remote_code=True)
streamer = TextStreamer(tokenizer, skip_special_tokens=True)

# Convert prompt to tokens
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"


def chat(text, max_new_tokens=512):
    prompt = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
        bos_token=tokenizer.bos_token,
        b_inst=B_INST,
        system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
        prompt=text,
        e_inst=E_INST,
    )

    tokens = tokenizer(
        prompt,
        return_tensors='pt'
    ).input_ids.cuda()

    # Generate output
    generation_output = model.generate(
        tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        max_new_tokens=max_new_tokens
    )

    return tokenizer.decode(generation_output[0])

Replacing layers...: 100% 32/32 [00:02<00:00, 11.64it/s]


CPU times: user 8.28 s, sys: 3.45 s, total: 11.7 s
Wall time: 8.65 s


In [19]:
%%time
print(chat("クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。"))

<s><s> [INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。 [/INST]  承知しました。以下がクマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説です。

クマは森の奥にある小屋で一人、暮らしていた。彼はいつも一人で過ごしていたが、今日は少し違った。彼は海辺に出かけたいと思ったのだ。

海辺までは結構長い道のりだったが、クマは元気だった。途中、蜂に刺されて少しだけ足を引きずることはあったが、それでもなんとか海辺に到着した。

海辺に着いたクマは、大きなためいていた。そしてその彼の前には、一匹のアザラシがいた。アザラシはクマが来たことに驚いていたが、すぐに彼の側に歩み寄った。

「あ、あなたはクマですね。私はアザラシです。よろしく」クマはそういうと、アザラシに対して手を差し伸べた。

「えっ、あの、私はクマなんですが……」アザラシは困惑したようにそう言った。

「そうなんだ。私はクマのことをずっと見ていて、ついに会えて嬉しいです」クマはそう言うと、アザラシの背中に手を回して抱きしめた。

「えっ
CPU times: user 4.84 s, sys: 4.06 ms, total: 4.84 s
Wall time: 4.84 s


In [3]:
%%time
print(chat("小学生向けの四則演算の問題を5問出してください。最初は問題だけ、次に解答のみをください。"))

<s><s> [INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

小学生向けの四則演算の問題を5問出してください。最初は問題だけ、次に解答のみをください。 [/INST]  承知しました。小学生向けの四則演算の問題を5問出し、順を追ってお答えします: 

問題: 567 - 234 = ?
問題: 172 - 56 = ?
問題: 345 + 216 = ?
問題: 789 - 345 = ?
問題: 456 - 234 = ?

解答: 333
128
261
545
423

以上、ご対応いたしました。</s>
CPU times: user 1.92 s, sys: 0 ns, total: 1.92 s
Wall time: 1.92 s


In [3]:
%%time
print(chat("豊臣秀吉について知っていることを教えてください。"))

<s><s> [INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

豊臣秀吉について知っていることを教えてください。 [/INST]  豊臣秀吉は、戦国時代の武将で、のちに徳川家康の家臣となり、4代将軍となる豊臣秀頼の父親です。

1536年、羽柴(はしはち)家臣として織田信長に仕えます。

1555年、織田信長の命で美濃国に攻め込み、斎藤道三の娘を娶います。

1556年、織田信長の命で大野惣治と戦い、敗れますが、織田信長の帰路、伊勢湾岸の海賊を討伐し、功を上げます。

1560年、織田信長により、四国征伐に従い、従業を称えられます。

1562年、織田信長の命で美濃国へ攻め込み、斎藤道三を討ち取ります。

1568年、織田信長により朝倉との戦いに従います。

1569年、織田信長の命で朝倉との戦いに従い、三度もの偵察にも出撃します。

1570年、織田信長の命で上洛し、義昭を迎えます。

1573年、織田信長により、桶狭間の戦いに討ち取られます。

以上が豊臣秀吉についての知っていることです。</s>
CPU times: user 4.95 s, sys: 1.84 ms, total: 4.95 s
Wall time: 4.95 s


In [10]:
%%time
# 素のTransformersで試す

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"
text = "クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。"

model_name = "elyza/ELYZA-japanese-Llama-2-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto")

if torch.cuda.is_available():
    model = model.to("cuda")
    
def chat2(text, max_new_tokens=512):
    prompt = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
        bos_token=tokenizer.bos_token,
        b_inst=B_INST,
        system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
        prompt=text,
        e_inst=E_INST,
    )


    with torch.no_grad():
        token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

        output_ids = model.generate(
            token_ids.to(model.device),
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
    return output

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 1min 56s, sys: 7.81 s, total: 2min 4s
Wall time: 1min 59s


In [12]:
%%time
print(chat2("小学生向けの四則演算の問題を5問出してください。最初は問題だけ、次に解答のみをください。"))

承知しました。小学生向けの四則演算の問題を5問お出しします。

1. 3×4=?
2. 5÷2=?
3. 7−3=?
4. 9×8=?
5. 10−5=?

次に、解答をお出しします。

1. 3×4=12
2. 5÷2=2
3. 7−3=4
4. 9×8=72
5. 10−5=5

これらはあくまで一例ですので、参考になれば幸いです。
CPU times: user 4.53 s, sys: 28.2 ms, total: 4.56 s
Wall time: 4.56 s
